In [1]:
import Models.JournalModel as JM
import pandas as pd

ej = JM.Journal('Economic Journal', "ej", "Data/Economic_Journal")

In [2]:

def scan_article(a):
    fail_list = []
    output_df = pd.DataFrame()
    table_id = 1
    headings = a.list_headings('list')
    for h in headings:
        if h.name.lower() in ['notes', 'appendix', 'references', 'supplementary data', 'author notes']:
            continue
        for t in h.list_tables('list'):
            try:
                cdf = t.list_coefficients()
                if isinstance(cdf, pd.DataFrame):
                    existing_cols = list(cdf.columns)
                    #cdf.insert(0, "Age", [21, 23, 24, 21], True)
                    cols_to_add = [
                        ('journal', a.journal().abbreviation),
                        ('volume', a.volume_number),
                        ('year', a.year),
                        ('issue', a.issue_number),
                        ('article_id', a.id),
                        ('article_doi', a.doi),
                        ('heading_id', h.id_global),
                        ('heading_name', h.name),
                        ('table_id', table_id ),
                        ('table_name', t.name)

                    ]
                    for c in cols_to_add:
                        cdf[c[0]] = c[1]
                    col_names_to_add = [x[0] for x in cols_to_add]
                    cdf = cdf.reindex(columns=(col_names_to_add + existing_cols))

                    if output_df.empty:
                        output_df = cdf
                    else:
                        output_df = output_df.append(cdf)
            except:
                # table could not be analyzed
                fail_list.append(table_id)

            table_id += 1

    return (output_df, fail_list)

#a = ej.find_article_by_doi("https://doi.org/10.1111/ecoj.12509")
#scan_article(a)


In [3]:
# study PLACEBO

# vol = ej.volume(128)
# iss = vol.issue(614)
# a = iss.article(6)
# tables = a.list_tables("list")
# for t in tables:
#     print(t.list_coefficients())


In [15]:
vol = ej.volume(128)
iss = vol.issue(614)
a = iss.article(7)
tables = a.list_tables("list")
t = tables[13]  # look at 1 (fixed now) 4 (se in brackets) 7, 9 (there is a column called z statistic)
print(t.footnotes)
t.df

# THis has to do with https://academic-oup-com.ludwig.lub.lu.se/ej/article/128/614/2414/5230962
# check sample of failed tables, and see if it has similar problem
# proposed fix: shift all header rows except first by prepending a blank td. first tr in thead etend rowspan on last td
# this i do because I assume that the first td of the first tr should be correct.

***p < 0.01, **p < 0.05, *p < 0.1. The standard errors were clustered at the Senate level.


,Variables,Marginal effects × 100,z‐statistic
0,Panel (a),Panel (a),Panel (a)
1,Alte Kämpfer,23.64***,4.626
2,WWII dummy,45.62***,11.62
3,Alte Kämpfer × WWII dummy,−21.92***,−4.136
4,Observations,3579,NaN
5,All other controls,Yes,NaN
6,Location fixed effects,Yes,NaN
7,Time fixed effects,No,NaN
8,Log likelihood,"−1,539.973",NaN
9,Pseudo R2,0.301,NaN


In [12]:
t.list_coefficients()

In [6]:
# scan a specific volume
output_df = pd.DataFrame()
vol = ej.volume(128)
iss = vol.issue(614)
for i in vol.list_issues('list'):
    iss = vol.issue(i)
    for a in iss.list_articles('list'):
        try:
            x = scan_article(a)
        except:
            print(f"FAILED ARTICLE {a.id}")
            continue
        df = x[0]
        fails = x[1]
        if df.empty == False:
            if output_df.empty:
                output_df = df
            else:
                output_df = output_df.append(df)

        if len(fails) == 0:
            print(f"COMPLETED ARTICLE {a.id}")
        else:
            print(f"COMPLETED ARTICLE {a.id}: FAILED TABLES {fails}")

output_df

COMPLETED ARTICLE 1: FAILED TABLES [3, 4]
COMPLETED ARTICLE 2
COMPLETED ARTICLE 3
COMPLETED ARTICLE 4
COMPLETED ARTICLE 5
COMPLETED ARTICLE 6
COMPLETED ARTICLE 7
COMPLETED ARTICLE 8
COMPLETED ARTICLE 9
COMPLETED ARTICLE 10: FAILED TABLES [11]
FAILED ARTICLE 11
COMPLETED ARTICLE 12
COMPLETED ARTICLE 13: FAILED TABLES [3]
COMPLETED ARTICLE 14
COMPLETED ARTICLE 1
COMPLETED ARTICLE 2
FAILED ARTICLE 3
FAILED ARTICLE 4
COMPLETED ARTICLE 5
COMPLETED ARTICLE 6
COMPLETED ARTICLE 7
COMPLETED ARTICLE 8
COMPLETED ARTICLE 9
COMPLETED ARTICLE 10
FAILED ARTICLE 11
COMPLETED ARTICLE 12
COMPLETED ARTICLE 13
COMPLETED ARTICLE 14
COMPLETED ARTICLE 1
COMPLETED ARTICLE 2: FAILED TABLES [13]
COMPLETED ARTICLE 3: FAILED TABLES [4]
COMPLETED ARTICLE 4
COMPLETED ARTICLE 5
FAILED ARTICLE 6
COMPLETED ARTICLE 7
COMPLETED ARTICLE 8
COMPLETED ARTICLE 9: FAILED TABLES [5, 6]
COMPLETED ARTICLE 10
COMPLETED ARTICLE 11
COMPLETED ARTICLE 12
COMPLETED ARTICLE 13: FAILED TABLES [4]
COMPLETED ARTICLE 14
COMPLETED ARTICLE 1

,journal,volume,year,issue,article_id,article_doi,heading_id,heading_name,table_id,table_name,coefficient,standard_error,row_id,row_name,col_id,col_name,sig_stars,z_score,p_value
0,ej,128,2018,608,1,https://doi.org/10.1111/ecoj.12429,5,4. Results,2,Table 1,-0.171,0.059,7,Female (dummy),2,(2) Attractiveness relative to surrounding areas,2,2.898305,0.003752
1,ej,128,2018,608,1,https://doi.org/10.1111/ecoj.12429,5,4. Results,2,Table 1,-0.006,0.066,7,Female (dummy),3,(3) Distinctiveness relative to surrounding areas,0,0.090909,0.927565
2,ej,128,2018,608,1,https://doi.org/10.1111/ecoj.12429,5,4. Results,2,Table 1,0.021,0.067,7,Female (dummy),4,(4) Distinctiveness relative to surrounding areas,0,0.313433,0.753952
3,ej,128,2018,608,1,https://doi.org/10.1111/ecoj.12429,5,4. Results,2,Table 1,-0.034,0.075,7,Female (dummy),5,(5) Attractiveness of buildings,0,0.453333,0.650309
4,ej,128,2018,608,1,https://doi.org/10.1111/ecoj.12429,5,4. Results,2,Table 1,-0.029,0.080,7,Female (dummy),6,(6) Attractiveness of buildings,0,0.362500,0.716978
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27,ej,128,2018,616,12,https://doi.org/10.1111/ecoj.12570,6,The Effect of Information on Mayors’ Decisions,10,Table 10,-45.400,19.600,4,1 year before election × post,4,Individual trends (4) Controls,2,2.316327,0.020540
28,ej,128,2018,616,12,https://doi.org/10.1111/ecoj.12570,6,The Effect of Information on Mayors’ Decisions,10,Table 10,-29.100,16.100,4,1 year after election × post,1,Baseline char. interactions (1) Trend,1,1.807453,0.070692
29,ej,128,2018,616,12,https://doi.org/10.1111/ecoj.12570,6,The Effect of Information on Mayors’ Decisions,10,Table 10,-30.100,16.300,4,1 year after election × post,2,Baseline char. interactions (2) Dummies,1,1.846626,0.064801
30,ej,128,2018,616,12,https://doi.org/10.1111/ecoj.12570,6,The Effect of Information on Mayors’ Decisions,10,Table 10,-18.200,15.400,4,1 year after election × post,3,Individual trends (3) Pre‐estimated,0,1.181818,0.237278


In [9]:
# scan all of the articles in EJ
fails = {'volume': [], 'issue': [], 'article': [], 'failed_all': [], 'tables': []}
failed_tables = []
output_df = pd.DataFrame()

for v in ej.list_volumes('list'):
    vol = ej.volume(v)

    for i in vol.list_issues('list'):
        iss = vol.issue(i)
        for a in iss.list_articles('list'):
            try:
                x = scan_article(a)
            except:
                print(f"FAILED ARTICLE {a.id}")
                fails['volume'].append(a.volume_number)
                fails['issue'].append(a.issue_number)
                fails['article'].append(a.doi)
                fails['failed_all'].append(1)
                fails['tables'].append(None)
                continue
            df = x[0]
            failed_tables = x[1]
            if df.empty == False:
                if output_df.empty:
                    output_df = df
                else:
                    output_df = output_df.append(df)

            if len(failed_tables) == 0:
                print(f"COMPLETED ARTICLE {a.id}")
            else:
                print(f"COMPLETED ARTICLE {a.id}: FAILED TABLES {failed_tables}")
                fails['volume'].append(a.volume_number)
                fails['issue'].append(a.issue_number)
                fails['article'].append(a.doi)
                fails['failed_all'].append(0)
                fails['tables'].append(failed_tables)


output_df



FAILED ARTICLE 1
COMPLETED ARTICLE 2: FAILED TABLES [4]
COMPLETED ARTICLE 3
COMPLETED ARTICLE 4
COMPLETED ARTICLE 5
COMPLETED ARTICLE 6
COMPLETED ARTICLE 7
COMPLETED ARTICLE 8
COMPLETED ARTICLE 9
COMPLETED ARTICLE 10
COMPLETED ARTICLE 11
COMPLETED ARTICLE 1
COMPLETED ARTICLE 2
COMPLETED ARTICLE 3
COMPLETED ARTICLE 4
COMPLETED ARTICLE 5
COMPLETED ARTICLE 6
COMPLETED ARTICLE 7
COMPLETED ARTICLE 8
COMPLETED ARTICLE 9
COMPLETED ARTICLE 10
COMPLETED ARTICLE 11
COMPLETED ARTICLE 12
COMPLETED ARTICLE 13
COMPLETED ARTICLE 14
COMPLETED ARTICLE 15
COMPLETED ARTICLE 16
COMPLETED ARTICLE 17
COMPLETED ARTICLE 18
COMPLETED ARTICLE 19
COMPLETED ARTICLE 20
COMPLETED ARTICLE 21
COMPLETED ARTICLE 22
COMPLETED ARTICLE 23
COMPLETED ARTICLE 24
COMPLETED ARTICLE 25
COMPLETED ARTICLE 1
COMPLETED ARTICLE 2
COMPLETED ARTICLE 3
COMPLETED ARTICLE 4
COMPLETED ARTICLE 5
COMPLETED ARTICLE 6: FAILED TABLES [1]
COMPLETED ARTICLE 7
COMPLETED ARTICLE 8
FAILED ARTICLE 9
COMPLETED ARTICLE 10
COMPLETED ARTICLE 11
COMPLETE

,journal,volume,year,issue,article_id,article_doi,heading_id,heading_name,table_id,table_name,coefficient,standard_error,row_id,row_name,col_id,col_name,sig_stars,z_score,p_value
1,ej,113,2003,484,2,https://doi.org/10.1111/1468-0297.00086,5,4. Results,3,Table 3,0.0030,1.00,6,Variance,1,Men Group A Independent,0,0.003000,0.997606
2,ej,113,2003,484,2,https://doi.org/10.1111/1468-0297.00086,5,4. Results,3,Table 3,0.0535,1.00,6,Variance,2,Men Group B Low wage,0,0.053500,0.957334
3,ej,113,2003,484,2,https://doi.org/10.1111/1468-0297.00086,5,4. Results,3,Table 3,0.0435,1.00,6,Variance,3,Men Group C High wage,0,0.043500,0.965303
4,ej,113,2003,484,2,https://doi.org/10.1111/1468-0297.00086,5,4. Results,3,Table 3,0.0047,1.00,6,Variance,4,Women Group A Independent,0,0.004700,0.996250
5,ej,113,2003,484,2,https://doi.org/10.1111/1468-0297.00086,5,4. Results,3,Table 3,0.0512,1.00,6,Variance,5,Women Group B Low wage,0,0.051200,0.959166
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11,ej,131,2021,640,13,https://doi.org/10.1093/ej/ueab046,3,2. An Application: Military Spending in US States,5,Table 5.,-0.1400,0.06,4,"MD(W,5)",4,"Guessed feedback variable |$y_{H,t}$|",0,2.333333,0.019631
12,ej,131,2021,640,13,https://doi.org/10.1093/ej/ueab046,3,2. An Application: Military Spending in US States,5,Table 5.,-0.2900,0.06,4,"MD(I,5)",1,"Guessed feedback variable |$l_{H,t-1}$|",0,4.833333,0.000001
13,ej,131,2021,640,13,https://doi.org/10.1093/ej/ueab046,3,2. An Application: Military Spending in US States,5,Table 5.,-0.1000,0.06,4,"MD(I,5)",2,"Guessed feedback variable |$l_{H,t}$|",0,1.666667,0.095581
14,ej,131,2021,640,13,https://doi.org/10.1093/ej/ueab046,3,2. An Application: Military Spending in US States,5,Table 5.,-0.2200,0.06,4,"MD(I,5)",3,"Guessed feedback variable |$y_{H,t-1}$|",0,3.666667,0.000246


In [12]:
# export the output to excel
writer = pd.ExcelWriter('output.xlsx',engine='xlsxwriter',options={'strings_to_urls': False})
output_df.to_excel(writer, index=False)
writer.close()


In [13]:
# export the fail list to excel
writer = pd.ExcelWriter('fails.xlsx',engine='xlsxwriter',options={'strings_to_urls': False})
pd.DataFrame(fails).to_excel(writer, index=False)
writer.close()

In [16]:
a = ej.find_article_by_doi("https://doi.org/10.1111/ecoj.12509")
a.list_tables()

,name,title
id,,
1,Table 1,Education Levels in the Experiment and Represe...
2,Table 2,Attrition at Endline (week 16)
3,Table 2,Attrition at Endline (week 16)
4,Table 3,Effects of Transport Subsidies on Employment O...
5,Table 4,Treatment Effects on Job Quality at Endline
6,Table 5,Persistence of Employment Effects at Second Fo...
7,Table 6,Persistence of Impacts on Search Behaviour
8,Table 7,Heterogeneous Effects by Poor/Non‐poor on Job ...
9,Table 8,Impact of the Phone Call Survey on Outcomes at...


In [17]:
a.table(3).df

Panel (b): Impact of treatment and calls on response (dependent variable: responded to endline survey)  \
                                                                                      Unnamed: 0_level_1   
                                                                                      Unnamed: 0_level_2   
0                                          Transport                                                       
1                                                NaN                                                       
2                                               Call                                                       
3                                                NaN                                                       
4                                       Observations                                                       

                                          
            (1)          (2)              
  Pooled sample Board sample City sample  
0        0.0021      −0.0058      0.0043  
1      (0.0330)     (0.0516)    (0.0414)  
2        0.0422       0.0380      0.0634  
3      (0.0355)     (0.0467)    (0.0584)  
4           877          877         877

In [18]:
a.table(3).list_coefficients()

,coefficient,standard_error,row_id,row_name,col_id,col_name,sig_stars,z_score,p_value
0,0.0021,0.0330,3,Transport,1,Panel (b): (1) Pooled sample,0,0.063636,0.949260
1,-0.0058,0.0516,3,Transport,2,Panel (b): (2) Board sample,0,0.112403,0.910504
2,0.0043,0.0414,3,Transport,3,Panel (b): (2) City sample,0,0.103865,0.917277
3,0.0422,0.0355,3,Call,1,Panel (b): (1) Pooled sample,0,1.188732,0.234545
4,0.0380,0.0467,3,Call,2,Panel (b): (2) Board sample,0,0.813704,0.415814
5,0.0634,0.0584,3,Call,3,Panel (b): (2) City sample,0,1.085616,0.277649


In [17]:
t = a.table(9)
print(f"{t.name}: {t.title}")
print(t.footnotes)
print(f"parentheses contain: {t.parentheses_unit}")
t.df

Table 8: Impact of the Phone Call Survey on Outcomes at Endline
Dependent variables are listed at the top of each column. Results are from OLS regressions on phone survey outcomes, with different treatment effects estimated as the average of groups of four weeks. Each coefficient gives the estimate for the treatment effect of transport with the sample restricted to the weeks denoted in the first column. The total number of observation used all regressions in each row is given in the last column (N). Standard errors are in parenthesis and are robust to correlation within clusters (subcities within Addis Ababa). *denotes significance at the 10%, ** at the 5% and *** at the 1% level
parentheses contain: standard error


,Unnamed: 0_level_0,(1),(2),(3),(4),(5)
,Unnamed: 0_level_1,Searchnow,Searchboards,Discouraged,Work,Work perm
0,Panel (a): Average impacts at endline (week 16),Panel (a): Average impacts at endline (week 16),Panel (a): Average impacts at endline (week 16),Panel (a): Average impacts at endline (week 16),Panel (a): Average impacts at endline (week 16),Panel (a): Average impacts at endline (week 16)
1,Transport,0.095*,0.084,−0.059*,0.051,0.031
2,NaN,(0.048),(0.054),(0.030),(0.045),(0.032)
3,Calls,−0.027,−0.0046,0.010,0.014,−0.0036
4,NaN,(0.050),(0.046),(0.044),(0.053),(0.033)
5,R2,0.021,0.073,0.037,0.016,0.047
6,Panel (b): Average impacts at endline (week 16...,Panel (b): Average impacts at endline (week 16...,Panel (b): Average impacts at endline (week 16...,Panel (b): Average impacts at endline (week 16...,Panel (b): Average impacts at endline (week 16...,Panel (b): Average impacts at endline (week 16...
7,Transport* boards,0.13*,0.11,−0.050,0.057,0.099**
8,NaN,(0.073),(0.090),(0.044),(0.060),(0.045)


In [18]:


t.list_coefficients()



,coefficient,standard_error,row_id,row_name,col_id,col_name,sig_stars,z_score,p_value
0,0.09500,0.048,2,Transport,1,(1) Searchnow,1,1.979167,0.047797
1,0.08400,0.054,2,Transport,2,(2) Searchboards,0,1.555556,0.119814
2,-0.05900,0.030,2,Transport,3,(3) Discouraged,1,1.966667,0.049222
3,0.05100,0.045,2,Transport,4,(4) Work,0,1.133333,0.257074
4,0.03100,0.032,2,Transport,5,(5) Work perm,0,0.968750,0.332670
5,-0.02700,0.050,4,Calls,1,(1) Searchnow,0,0.540000,0.589197
6,-0.00460,0.046,4,Calls,2,(2) Searchboards,0,0.100000,0.920344
7,0.01000,0.044,4,Calls,3,(3) Discouraged,0,0.227273,0.820212
8,0.01400,0.053,4,Calls,4,(4) Work,0,0.264151,0.791664
9,-0.00360,0.033,4,Calls,5,(5) Work perm,0,0.109091,0.913130


In [4]:
a2 = ej.find_article_by_doi("https://doi.org/10.1111/ecoj.12550")
t2 = a2.table(4)
#t2.df
print(t2.footnotes)
print(t2.parentheses_unit)
t2.df

Notes. Columns (1)–(5) cut the sample by the indicated minimum percentile of income using both homeowners and renters. Columns (6)–(9) cut the sample by homeownership status. The reason to allow for overlapping groups is that a policymaker is likely to care about the average effect among all high income households rather than a single percentile. Results for income above the 95th percentile can be seen in Table 3. Column (6) focuses on homeowners and column (7) focuses on renters above the 75th percentile. Column (8) focuses on homeowners and column (9) focuses on renters above the 90th percentile. The dependent variable is the average commute time per worker in a household, the variable R is one when the MSA has a reciprocity agreement in place, mtr is the marginal tax rate paid, and ▵atr is the average tax rate in the state of residence minus the counterfactual average tax rate. All specifications include MSA by year dummies, state by year dummies, occupation dummies, household and p

,Income percentile,(1),(2),(3),(4),(5),(6),(7),(8),(9)
,Income percentile,Income heterogeneity,Income heterogeneity,Income heterogeneity,Income heterogeneity,Income heterogeneity,Homeownership,Homeownership,Homeownership,Homeownership
,Unnamed: 0_level_2,>50,>75,>80,>85,>90,>75 & homeowner,>75 & renter,>90 & homeowner,>90 & renter
0,▵atr,0.468**,0.826***,0.963***,1.112***,1.236***,0.682***,1.225***,1.047***,1.271***
1,NaN,(0.191),(0.278),(0.273),(0.299),(0.310),(0.244),(0.371),(0.279),(0.297)
2,▵atr × R,−0.561*,−1.006**,−1.032**,−1.383***,−2.071***,−0.834**,−0.415,−1.808***,−3.024*
3,NaN,(0.313),(0.434),(0.437),(0.503),(0.542),(0.392),(1.002),(0.473),(1.740)
4,mtr,0.087**,0.121*,0.131**,0.108**,0.052,0.120**,0.028,0.069**,−0.082
5,NaN,(0.043),(0.067),(0.060),(0.048),(0.045),(0.060),(0.126),(0.031),(0.112)
6,mtr × R,−0.030,−0.104,−0.120*,−0.106,−0.079,−0.101,−0.107,−0.086*,−0.014
7,NaN,(0.044),(0.069),(0.068),(0.070),(0.051),(0.067),(0.147),(0.046),(0.171)


In [5]:
t2.list_coefficients()

,coefficient,standard_error,row_id,row_name,col_id,col_name,sig_stars,z_score,p_value
0,0.468,0.191,1,▵atr,1,(1) Income heterogeneity >50,2,2.450262,0.014275
1,0.826,0.278,1,▵atr,2,(2) Income heterogeneity >75,3,2.971223,0.002966
2,0.963,0.273,1,▵atr,3,(3) Income heterogeneity >80,3,3.527473,0.000420
3,1.112,0.299,1,▵atr,4,(4) Income heterogeneity >85,3,3.719064,0.000200
4,1.236,0.310,1,▵atr,5,(5) Income heterogeneity >90,3,3.987097,0.000067
5,0.682,0.244,1,▵atr,6,(6) Homeownership >75 & homeowner,3,2.795082,0.005189
6,1.225,0.371,1,▵atr,7,(7) Homeownership >75 & renter,3,3.301887,0.000960
7,1.047,0.279,1,▵atr,8,(8) Homeownership >90 & homeowner,3,3.752688,0.000175
8,1.271,0.297,1,▵atr,9,(9) Homeownership >90 & renter,3,4.279461,0.000019
9,-0.561,0.313,3,▵atr × R,1,(1) Income heterogeneity >50,1,1.792332,0.073080


In [6]:
# add warnings to the table object